In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import torch

from umap.umap_ import UMAP
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
from tqdm import tqdm

In [20]:
class VectorStore:
    def __init__(self, model, size):
        self.documents = []
        self.embeddings = np.empty(size)
        self.model = model  

    def add_to_store(self, document):
        self.documents.append(document)
        embedding = self.model.encode(document, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False, batch_size=32)
        self.embeddings = np.vstack((self.embeddings, embedding))

In [21]:
def build_scatter_2d(data, projections, title):
    fig = px.scatter(projections,
                x=0, 
                y=1,
                color=data['labels'], 
                labels={'color': 'category'},
                template='plotly_dark',
                height=700,
                )

    fig.update_layout(
                title={'text': title,
                        'y':0.95,
                        'x':0.5},
                showlegend=True)

    fig.show()

In [22]:
df = pd.read_csv('./data/part_1_BIG_MODELfor_merge.csv')

In [23]:
df2 = pd.read_csv('./data/part_2_for_merge.csv')

In [24]:
df = df.join(df2)

In [25]:
df = df.dropna(subset=['product', 'location', 'description']).reset_index(drop=True)

In [26]:
df

,location,product,tender_number,tender_name,tender_company,quantity,description,requirements,stages,delivery_time,start_date,end_date
0,"Гуандун, Чжунцин, Каннинг",оборудование для замены коробок,CGN-202305310008,NaN,Новая энергетическая компания,NaN,"Проект <<Хиннин>>, расположенный в городе Цзян...",Этот тендер будет состоять из первого сегмента...,четыре сегмента: 1-й пункт: кузов; 2-й участок...,Предварительный 15 августа 2023 года; срок око...,15.08.2023,NaN
1,"Немонгольский автономный район, УНИТА, правый ...",оборудование для оптических камер,CGN-202211140006,NaN,Новая энергетическая компания,NaN,Закупка оборудования для фотоаппаратов УНИТА в...,Участники торгов предоставили три комплекта ка...,2,до 10 июня 2023 года,NaN,NaN
2,"Синьцзян-Уйгурский автономный район и Тэгу, ок...",1 млн. кВт оптико-волнового поля PC,CGN-202307050014,NaN,China Solar Energy Successing Ltd.,NaN,Общий контрактный проект PC в 1 млн кВт-диапаз...,Упаковка 1: первый этап проекта генерального п...,четыре сегмента,Планируемый период работы составляет с 15 авгу...,15.08.2023,30.12.2023
3,"Хубэй, Хуанган, округ Чун",гидроэнергетическая станция,CGN-202301310002,NaN,Чайна нью-Энтерпрайз,NaN,Проект &lt; &lt; Старый ветер в Синьцзян-Тауне...,Содержание и сфера охвата данного тендера вклю...,347 календарных дней,347 календарных дней,28.02.2023,10.02.2024
4,"Пекин, Тойота",сбор фотоэлектрических компонентов,CGN-202307040015,NaN,Чайна Хинэ,NaN,С 2023 по 2024 гг. были собраны матрицы фотоэл...,Данный тендерный проект разделен на семь пункт...,7,NaN,2023,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
380,"Автономный район Гуанси-Маньчжур, муниципалите...",ветрогенераторное оборудование,CGN-20221220004,NaN,NaN,NaN,На юго-западе юго-западного района Порт-Куньси...,В рамках этого тендера были закуплены ветряное...,"Вместимость проекта составляет 50 МВт, а общая...",20.05.2023,20.05.2023,20.05.2023
381,Гуанси-Шань; город Порт-оф-Си; порт,материалы для комбинированной модификации гене...,CGN-202301180008,NaN,Хинси энтерпрайз,NaN,атомная электростанция порта Гуанси находится ...,"данный тендерный проект разделен на 1 пункт, к...",1,15.07.2023,15.07.2023,31.12.2025
382,"Цинго, округ Цинго, провинция Хэйлунцзян, Цзин...",оборудование для преобразования коробок,CGN-202305310011,NaN,Киото,NaN,новая энергетическая ветряная площадка &lt; &l...,4-й пункт: «Геронцзян Юнг». Поставки включают ...,4,20.08.2023,20.08.2023,NaN
383,Гуандун,Проектные блоки LOT88Ea & LOT88Ea для борьбы с...,CGN-20230103001,NaN,NaN,NaN,&lt; &lt; Ядерная энергетика Китая и Китая &gt...,сейф для борьбы с пожарными клапанами ядерного...,NaN,1 сентября 2023 года,NaN,NaN


In [27]:
model_name = '../models/embaas_sentence-transformers-e5-large-v2/'

In [28]:
device = torch.device("mps")
device

device(type='mps')

In [29]:
model = SentenceTransformer(model_name, device=device)
model[1].pooling_mode_mean_tokens = False
model[1].pooling_mode_cls_token = True
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [30]:
store_product = VectorStore(model, size=(0, model[1].word_embedding_dimension)) 
for i in tqdm(range(df.shape[0])):
    store_product.add_to_store(df['product'][i])

100%|██████████| 385/385 [01:19<00:00,  4.86it/s]


In [31]:
store_product.embeddings.shape

(385, 1024)

In [61]:
umap_model = UMAP(n_neighbors=5, min_dist=0.3, metric='correlation', random_state=42, n_components=2)
projections = umap_model.fit_transform(store_product.embeddings)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [62]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(projections)
labels = kmeans.labels_
data_labeled = np.column_stack((projections, labels))
temp_df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
temp_df['labels'] = temp_df['labels'].astype('str')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [63]:
build_scatter_2d(temp_df, projections, 'Product: K-means / UMAP')

In [64]:
df['product_cluster'] = labels

location

In [37]:
store_location = VectorStore(model, size=(0, model[1].word_embedding_dimension)) 
for i in tqdm(range(df.shape[0])):
    store_location.add_to_store(df['location'][i])

100%|██████████| 385/385 [00:16<00:00, 23.69it/s]


In [38]:
store_location.embeddings.shape

(385, 1024)

In [65]:
umap_model = UMAP(n_neighbors=5, min_dist=0.3, metric='correlation', random_state=42, n_components=2)
projections = umap_model.fit_transform(store_location.embeddings)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/umap/spectral.py:550: UserWarning:

Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!



In [66]:
kmeans = KMeans(n_clusters=30)
kmeans.fit(projections)
labels = kmeans.labels_
data_labeled = np.column_stack((projections, labels))
temp_df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
temp_df['labels'] = temp_df['labels'].astype('str')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [67]:
build_scatter_2d(temp_df, projections, 'Location: K-means / UMAP')

In [68]:
df['location_cluster'] = labels

In [50]:
store_descrition = VectorStore(model, size=(0, model[1].word_embedding_dimension)) 
for i in tqdm(range(df.shape[0])):
    store_descrition.add_to_store(df['description'][i])

100%|██████████| 385/385 [00:16<00:00, 24.05it/s]


In [69]:
umap_model = UMAP(n_neighbors=5, min_dist=0.3, metric='correlation', random_state=42, n_components=2)
projections = umap_model.fit_transform(store_descrition.embeddings)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [70]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(projections)
labels = kmeans.labels_
data_labeled = np.column_stack((projections, labels))
temp_df = pd.DataFrame(data_labeled, columns=['X', 'Y', 'labels'])
temp_df['labels'] = temp_df['labels'].astype('str')

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [71]:
build_scatter_2d(temp_df, projections, 'Description: K-means / UMAP')

In [72]:
df['detailed_cluster'] = labels

In [75]:
df.to_csv('.result_with_clusters.csv', index=False)